In [7]:
import numpy as np
import pandas as pd
import jieba
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('entity/chinese.txt')

In [57]:
v1 = model["于"]
v2 = model["程"]
model.most_similar([v1 + v2])

KeyError: "word '程' not in vocabulary"

In [4]:

courses = ['chinese', 'math', 'physics', 'chemistry', 'biology', 'politics', 'history', 'geo']
for course in courses: 
  f = open('name/' + course + ".txt" , 'r')
  all_text = f.readlines()
  all_text = [e.strip() for e in all_text] 
  all_text = [''.join(e.split(' ')) for e in all_text]
  seg_list = [list(jieba.cut(text)) for text in all_text]
  entity2vec_dict = {}
  for i, seg in enumerate(seg_list):
    cnt = 0
    cur_vec = np.zeros(300)
    for text in seg:
      if text not in model.vocab:
        continue
      else:
        cnt += 1
        cur_vec += model[text]
    if cnt == 0:
      continue 
    entity2vec_dict[all_text[i]] = cur_vec / cnt
  def save_word2vec_format(fname, vocab, vector_size, binary=True):
      """Store the input-hidden weight matrix in the same format used by the original
      C word2vec-tool, for compatibility.

      Parameters
      ----------
      fname : str
          The file path used to save the vectors in.
      vocab : dict
          The vocabulary of words.
      vector_size : int
          The number of dimensions of word vectors.
      binary : bool, optional
          If True, the data wil be saved in binary word2vec format, else it will be saved in plain text.


      """
      
      total_vec = len(vocab)
      with open(fname, 'ab') as fout:
          print(total_vec, vector_size)
          # fout.write(gensim.utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
          # store in sorted order: most frequent words at the top
          for word, row in tqdm(vocab.items()):
              if binary:
                  row = row.astype(np.float32)
                  fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
              else:
                  fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))
  save_word2vec_format('entity/chinese.txt', entity2vec_dict, 300, False)